**Tropical Cyclone Freddy**


**How did Pangu-AI-forecast perform on predicting Freddy using ERA5-data as a comparison?**
- we have Pangu data initialized: 14.02, 19.02, 23.02

**Aim of this notebook**
- load both Pangu and ERA5 datasets
- save all Pangu initialisations in one .nc file
- calculate wind speed metrics
- convert pressure measurements
- crop to the lat-lon range of interest
- save the data in the /data/processed/ folder

**Import libraries**

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import xarray as xr
from datetime import datetime
import sys
import warnings
from glob import glob
import netCDF4 as nc
import h5netcdf
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.dates import DateFormatter, HourLocator
import datetime as dt
import matplotlib.dates as mdates

**Load the data**

In [2]:
## ERA5
era =  xr.open_dataset('../data/raw/ERA5/ERA_data_TC_Freddy2.nc')

## Pangu in one dataset
all_files=sorted(glob('../data/raw/Pangu/TC_FREDD*'))
datasets = []
for file in all_files:
    datasets.append(xr.open_dataset(file))

# concenating over the dimension time
pangu = xr.concat(datasets, dim='time')  

**Calculate winds speed at 850/500hPa level and at surface level**

In [3]:
### 850/500 hPA
# era
era = era.assign(wind_speed = lambda era: (era.u**2 + era.v**2)**0.5)
era['wind_speed'].attrs['GRIB_name'] = 'Wind speed'

# pangu
pangu = pangu.assign(wind_speed = lambda pangu: (pangu.u**2 + pangu.v**2)**0.5)
pangu['wind_speed'].attrs['GRIB_name'] = 'Wind speed'
pangu['msl'].attrs['GRIB_name'] = 'Mean sea level pressure'

### Surface level
# era 
era = era.assign(wind_speed_sfc = lambda era: (era.u10**2 + era.v10**2)**0.5)
era['wind_speed_sfc'].attrs['GRIB_name'] = 'Surface wind speed'
# pangu 

pangu = pangu.assign(wind_speed_sfc = lambda pangu: (pangu.u10**2 + pangu.v10**2)**0.5)
pangu['wind_speed_sfc'].attrs['GRIB_name'] = 'Surface wind speed'

**Crop the data to lat 10 - 70**

In [4]:
era_cropped = era.where((era.lon >= 25) & (era.lon <= 65), drop=True)
pangu_cropped = pangu.where((pangu.longitude >= 25) & (pangu.longitude <= 65), drop=True)

**Convert Pa to hPa**

In [5]:
era_cropped['msl'] = era_cropped['msl'] / 100
pangu_cropped['msl'] = pangu_cropped['msl'] / 100
pangu_cropped['msl'].attrs['GRIB_name'] = 'Mean sea level pressure'

**Convert units for q**

In [6]:
era_cropped = era_cropped.assign(q = era_cropped.q * 1000)
pangu_cropped = pangu_cropped.assign(q=pangu_cropped.q * 1000)
pangu_cropped['q'].attrs['GRIB_name'] = 'Specific humidity'

**Save the processed data**

In [9]:
# Save ERA5 cropped dataset
era_cropped.to_netcdf("../data/processed/ERA_processed_TC_Freddy.nc", mode='w')

# Save Pangu cropped dataset
pangu_cropped.to_netcdf("../data/processed/Pangu_processed_TC_Freddy.nc", mode='w')